In [2]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
from pathlib import Path

In [3]:
#Selecionando as Variáveis
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

Colunas que vai precisar pro cálculo:

LOTAÇÃO	MATRICULA	GRUPO	CRITERIO DE APOSENTADORIA	SEXO	DTNASC	DTADM	DATACARGO	SALPART	CARREIRA	TSANT	CONJUGE	DTNASCONJ	DTNASDEPINV	DTNASDEPVAL

In [67]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_INV'] = pd.to_datetime(bd_ativos['DT_NASC_INV'], yearfirst=True, format='%d/%m/%Y')

bd_ativos['DT_BASE'] = '31/12/2020'
bd_ativos['DT_BASE'] = pd.to_datetime(bd_ativos.DT_BASE, yearfirst=True, format='%d/%m/%Y')

In [ ]:
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)
statis_ativos.IDADE_CONJUGE = ano_dt_base - statis_ativos.IDADE_CONJUGE.dt.strftime('%Y').astype(int)
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5) | (statis_ativos.IDADE_CONJUGE > 127), 'IDADE_CONJUGE'] = None
statis_ativos.IDADE_CONJUGE

In [78]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = round((bd_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_CONJUGE = round((bd_ativos.DT_BASE - statis_ativos.IDADE_CONJUGE) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades dos Cônjuges | Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5) | (statis_ativos.IDADE_CONJUGE > 127), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_EMT'] = round(statis_ativos.IDADE_ADM - (bd_ativos['NU_TEMPO_RGPS']/365.25)) #Calculando a Idade de Entrada no Mercado de Trabalho | Idade de Admissão - (TSANT/365.25)
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos.IDADE_CONJUGE - statis_ativos.IDADE_SERVIDOR #Calculando a Diferença Etária do Cônjuge | Idade do Cônjuge - Idade do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos.DT_NASC_NOVO.mask(bd_ativos.DT_NASC_NOVO.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_NOVO = round((bd_ativos.DT_BASE - statis_ativos.IDADE_NOVO) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Mais Novo | Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos.IDADE_NOVO > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_INV'] = bd_ativos.DT_NASC_INV.mask(bd_ativos.DT_NASC_INV.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_INV = round((bd_ativos.DT_BASE - statis_ativos.IDADE_INV) / timedelta(days=365.25) - 0.5).astype(int) #Calculando as Idades do Filho Inválido | Data-Base - Data de Nascimento do Filho Inválido
statis_ativos.loc[statis_ativos.IDADE_INV > 127, 'IDADE_INV'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['TP_PREFEITURA'] = statis_ativos.IDADE_SERVIDOR - statis_ativos.IDADE_ADM #Calculando o Tempo de Prefeitura | Idade do Servidor - Idade de Admissão
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIFERIMENTO'] = bd_ativos.DT_PROV_APOSENT - statis_ativos.IDADE_SERVIDOR #Calculando o Diferimento até a Aposentadoria | Idade Provável de Aposentadoria - Idade do Servidor


#Idade de entrada no plano ou RPPS
#Perguntar a rúniu sobre a IDADE_EMT se considera o tempo do RPPS também

statis_ativos.head(20)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_EMT,DIF_ETARIA,IDADE_NOVO,IDADE_INV,TP_PREFEITURA,DIFERIMENTO
0,40,20,NaN,20.0,NaN,16.0,NaN,20,20
1,45,42,48.0,25.0,3.0,NaN,NaN,3,17
2,48,27,NaN,25.0,NaN,18.0,NaN,21,14
3,31,23,NaN,23.0,NaN,NaN,NaN,8,31
4,42,35,NaN,25.0,NaN,NaN,NaN,7,23
5,51,29,NaN,25.0,NaN,NaN,NaN,22,9
6,40,22,53.0,22.0,13.0,NaN,NaN,18,21
7,40,34,NaN,25.0,NaN,11.0,NaN,6,22
8,45,41,NaN,24.0,NaN,NaN,NaN,4,20
9,41,30,NaN,25.0,NaN,NaN,NaN,11,24


In [77]:
statis_ativos.IDADE_EMT.sum()

8699.0

In [64]:
statis_ativos[statis_ativos['IDADE_CONJUGE'].notnull()]

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_NOVO,TP_PREFEITURA,DIF_ETARIA
1,45,42,48.0,NaN,3,3.0
6,40,22,53.0,NaN,18,13.0
11,41,26,44.0,NaN,15,3.0
17,53,30,30.0,3.0,22,-23.0
23,40,28,54.0,16.0,12,14.0
24,32,21,36.0,13.0,11,4.0
31,48,37,38.0,15.0,11,-10.0
38,48,26,40.0,5.0,22,-8.0
41,26,25,52.0,16.0,0,26.0
42,36,26,43.0,15.0,10,7.0


In [57]:
bd_ativos['DT_NASC_CONJUGE'].count()#head(20)

47

In [8]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#statis_ativos['IDADE_ADM'] =  bd_ativos['DT_ING_ENTE'].dt.strftime('%Y').astype(int) - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].mask(bd_ativos['DT_NASC_CONJUGE'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_CONJUGE'] = ano_dt_base - statis_ativos['IDADE_CONJUGE'].dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 1) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 3) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 4) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos['DT_NASC_NOVO'].mask(bd_ativos['DT_NASC_NOVO'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_NOVO'] = ano_dt_base - statis_ativos['IDADE_NOVO'].dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos['IDADE_NOVO'] > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos['IDADE_CONJUGE'] - statis_ativos['IDADE_SERVIDOR'] #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

,IDADE_SERVIDOR,IDADE_ADM,IDADE_CONJUGE,IDADE_NOVO,DIF_ETARIA
0,40,20,NaN,16.0,NaN
1,45,43,48.0,NaN,3.0
2,48,27,NaN,18.0,NaN
3,31,23,NaN,NaN,NaN
4,42,35,NaN,NaN,NaN
5,51,29,NaN,NaN,NaN
6,40,22,53.0,NaN,13.0
7,40,34,NaN,11.0,NaN
8,45,42,NaN,NaN,NaN
9,41,30,NaN,NaN,NaN


In [ ]:
statis_ativos['IDADE_ADM_ARRED'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int)
statis_ativos['IDADE_ADM_SEM_ARRED'] = (bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['IDADE_ADM_ARRED_CIMA'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) + 0.5).astype(int)
statis_ativos['IDADE_ADM_ARRED_BAIXO'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int)
statis_ativos[['IDADE_ADM_ARRED', 'IDADE_ADM_SEM_ARRED', 'IDADE_ADM_ARRED_CIMA', 'IDADE_ADM_ARRED_BAIXO']]

In [ ]:
statis_ativos.IDADE_ADM3.sum()